<a href="https://colab.research.google.com/github/FreeeSoul/realFastCampus/blob/main/4_ch1_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [4]:
print(tf.__version__)

2.8.2


In [5]:
batch_size = 120
# batch 개수 의미 --> 몇 번 학습 마다 데이터 새로 가져올지.. 예비모의고사 10회 풀 때도
#10회를 한꺼번에 쭉 풀고 전부 한꺼번에 채점하는거보다 3회풀고 채점 후 재정비하고 다시 3회풀고 이러는게
#점수 더 잘나옴
epochs = 100
num_classes = 10

우리가 할 것 : 0~9까지 손으로 쓴 데이터를 가져온다음에 학습시킨 후 우리가 직접 손으로 쓴 글씨 인식하기

In [6]:
mnist = keras.datasets.mnist
# mnist는 데이터를 가져와서 우리가 일일히 정제할 필요없이 트레이닝, 테스트셋 모두 존재
#하므로 그거 가져와서 쓰면됨.
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [7]:
len(train_images), len(test_images)

(60000, 10000)

In [8]:
# input data에 대해 normalize를 해준다 0~1 사이로 --> 텐서플로우가 실수에서 동작을 주로 시키기
#때문에
train_images = train_images / 255.0
test_images = test_images / 255.0

In [9]:
# 모델 정의
model = keras.Sequential([ # 이렇게 리스트 안에 레이어들을 넣어준다., 
                          #리스트 형태로 만들면 자동으로 ★리니어 스택 형태가 된다.★
                         keras.layers.Flatten(input_shape=(28, 28)),
                          keras.layers.Reshape(target_shape=(28, 28, 1)),
                          keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=tf.nn.relu),
                          keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=tf.nn.relu),
                          keras.layers.MaxPooling2D(pool_size=(2, 2)),
                          keras.layers.Dropout(0.25),
                          keras.layers.Flatten(input_shape=(28, 28)),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(num_classes, activation='softmax')
                          # tf.nn.softmax로 써도 됨
])
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# 지금 mnist문제는 Multi Classification 문제라서 sparse_categorical_crossentropy를 Cost 함수
#형성 방식으로 사용

In [14]:
# 모델을 저장해보자~~~~
filepath = '/model/digits_model.h5'
modelCheckPoint = tf.keras.callbacks.ModelCheckpoint(filepath=filepath, save_best_only=True)

In [15]:
history = model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=epochs, batch_size=batch_size, callbacks=[modelCheckPoint])

Epoch 1/100
500/500 [==============================] - 4s 8ms/step - loss: 0.0518 - accuracy: 0.9839 - val_loss: 0.0405 - val_accuracy: 0.9869
Epoch 2/100
500/500 [==============================] - 4s 8ms/step - loss: 0.0462 - accuracy: 0.9857 - val_loss: 0.0345 - val_accuracy: 0.9888
Epoch 3/100
500/500 [==============================] - 4s 8ms/step - loss: 0.0419 - accuracy: 0.9865 - val_loss: 0.0324 - val_accuracy: 0.9901
Epoch 4/100
500/500 [==============================] - 4s 8ms/step - loss: 0.0397 - accuracy: 0.9874 - val_loss: 0.0313 - val_accuracy: 0.9892
Epoch 5/100
500/500 [==============================] - 4s 8ms/step - loss: 0.0371 - accuracy: 0.9882 - val_loss: 0.0330 - val_accuracy: 0.9894
Epoch 6/100
500/500 [==============================] - 4s 8ms/step - loss: 0.0338 - accuracy: 0.9892 - val_loss: 0.0303 - val_accuracy: 0.9901
Epoch 7/100
500/500 [==============================] - 5s 9ms/step - loss: 0.0322 - accuracy: 0.9893 - val_loss: 0.0275 - val_accuracy: 0.9902

In [12]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_loss, test_acc)
# 테스트 값들로 평가 진행 -> loss, accuracy 값 확인 가능

313/313 [==============================] - 2s 5ms/step - loss: 0.0386 - accuracy: 0.9874
0.03857832029461861 0.9873999953269958


In [16]:
def download(path):
  try:
    from google.colab import files
    files.download(path)
  except ImportError:
    import os
    print('Error download', os.path.join(os.getcwd(), path))

In [17]:
download(filepath)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
savefile = 'saved_digits.h5'
model.save(savefile)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                        

In [20]:
!ls -al

total 1364
drwxr-xr-x 1 root root    4096 Jul 23 02:37 .
drwxr-xr-x 1 root root    4096 Jul 23 02:16 ..
drwxr-xr-x 4 root root    4096 Jul 13 13:42 .config
drwx------ 5 root root    4096 Jul 23 02:13 drive
drwxr-xr-x 1 root root    4096 Jul 13 13:43 sample_data
-rw-r--r-- 1 root root 1375344 Jul 23 02:37 saved_digits.h5


In [21]:
# 저장했떤 모델을 로드해보자
load_model = tf.keras.models.load_model(savefile)
load_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                        